# Creating a Sentiment Analysis Web App
## Using PyTorch and SageMaker

_Deep Learning Nanodegree Program | Deployment_

---

Construct a complete project from end to end. The goal will be to have a simple web page which a user can use to enter a movie review. The web page will then send the review off to our deployed model which will predict the sentiment of the entered review.


## General Outline


1. Download or otherwise retrieve the data.
2. Process / Prepare the data.
3. Upload the processed data to S3.
4. Train a chosen model.
5. Test the trained model (typically using a batch transform job).
6. Deploy the trained model.
7. Use the deployed model.


In [ ]:
# Make sure that we use SageMaker 1.x
!pip install sagemaker==1.72.0

  Using cached sagemaker-1.72.0-py2.py3-none-any.whl
  Using cached smdebug_rulesconfig-0.1.4-py2.py3-none-any.whl (10 kB)
  Attempting uninstall: smdebug-rulesconfig
    Found existing installation: smdebug-rulesconfig 1.0.1
    Uninstalling smdebug-rulesconfig-1.0.1:
      Successfully uninstalled smdebug-rulesconfig-1.0.1
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.31.1
    Uninstalling sagemaker-2.31.1:
      Successfully uninstalled sagemaker-2.31.1


## Step 1: Downloading the data

As in the XGBoost in SageMaker notebook, we will be using the [IMDb dataset](http://ai.stanford.edu/~amaas/data/sentiment/)

> Maas, Andrew L., et al. [Learning Word Vectors for Sentiment Analysis](http://ai.stanford.edu/~amaas/data/sentiment/). In _Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies_. Association for Computational Linguistics, 2011.

In [ ]:
%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

mkdir: cannot create directory ‘../data’: File exists
--2021-04-08 09:18:14--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘../data/aclImdb_v1.tar.gz’

../data/aclImdb_v1. 100%[===================>]  80.23M  24.0MB/s    in 4.2s    

2021-04-08 09:18:18 (18.9 MB/s) - ‘../data/aclImdb_v1.tar.gz’ saved [84125825/84125825]



## Step 2: Preparing and Processing the data

Also, as in the XGBoost notebook, we will be doing some initial data processing.

In [ ]:
import os
import glob

def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [ ]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

IMDB reviews: train = 12500 pos / 12500 neg, test = 12500 pos / 12500 neg


Now that we've read the raw training and testing data from the downloaded dataset, we will combine the positive and negative reviews and shuffle the resulting records.

In [ ]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [ ]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

IMDb reviews (combined): train = 25000, test = 25000


Now that we have our training and testing sets unified and prepared, we should do a quick check and see an example of the data our model will be trained on. 

In [ ]:
print(train_X[100])
#print(train_y[100])

I gotta say, Clive Barker's Undying is by far the best horror game to have ever been made. I've played Resident Evil, Silent Hill and the Evil Dead and Castlevania games but none of them have captured the pure glee with which this game tackles its horrific elements. Barker is good at what he does, which is attach the horror to our world, and it shows as his hand is clearly everywhere in this game. Heck, even his voice is in the game as one of the main characters. Full of lush visuals and enough atmosphere to shake a stick at, Undying is the game to beat in my books as the best horror title. I just wish that this had made it to a console system but alas poor PC sales nipped that one in the bud.


The first step in processing the reviews is to make sure that any html tags that appear should be removed.

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup

def review_to_words(review):
    nltk.download("stopwords", quiet=True)
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

The `review_to_words` method defined above uses `BeautifulSoup` to remove any html tags that appear and uses the `nltk` package to tokenize the reviews. As a check to ensure we know how everything is working, try applying `review_to_words` to one of the reviews in the training set.

In [ ]:
# TODO: Apply review_to_words to a review (train_X[100] or any other review)
output = review_to_words(train_X[100])
print(output)

['gotta', 'say', 'clive', 'barker', 'undi', 'far', 'best', 'horror', 'game', 'ever', 'made', 'play', 'resid', 'evil', 'silent', 'hill', 'evil', 'dead', 'castlevania', 'game', 'none', 'captur', 'pure', 'glee', 'game', 'tackl', 'horrif', 'element', 'barker', 'good', 'attach', 'horror', 'world', 'show', 'hand', 'clearli', 'everywher', 'game', 'heck', 'even', 'voic', 'game', 'one', 'main', 'charact', 'full', 'lush', 'visual', 'enough', 'atmospher', 'shake', 'stick', 'undi', 'game', 'beat', 'book', 'best', 'horror', 'titl', 'wish', 'made', 'consol', 'system', 'ala', 'poor', 'pc', 'sale', 'nip', 'one', 'bud']


The method below applies the `review_to_words` method to each of the reviews in the training and testing datasets. In addition it caches the results.

In [ ]:
import pickle

cache_dir = os.path.join("../cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Preprocess training and test data to obtain words for each review
        #words_train = list(map(review_to_words, data_train))
        #words_test = list(map(review_to_words, data_test))
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [ ]:
# Preprocess data
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

Read preprocessed data from cache file: preprocessed_data.pkl


In [ ]:
len(test_X)

25000

### Create a word dictionary

To begin with, we need to construct a way to map words that appear in the reviews to integers. Here we fix the size of our vocabulary (including the 'no word' and 'infrequent' categories) to be `5000` but you may wish to change this to see how it affects the model.

> Complete the implementation for the `build_dict()` method below. Note that even though the vocab_size is set to `5000`, we only want to construct a mapping for the most frequently appearing `4998` words. This is because we want to reserve the special labels `0` for 'no word' and `1` for 'infrequent word'.

In [ ]:
import numpy as np

def build_dict(data, vocab_size = 5000):
    """Construct and return a dictionary mapping each of the most frequently appearing words to a unique integer."""
    
    # TODO: Determine how often each word appears in `data`. Note that `data` is a list of sentences and that a
    #       sentence is a list of words.
    
    word_count = {} # A dict storing the words that appear in the reviews along with how often they occur
    
    wordlist = []        # all words in a single list
    [wordlist.append(word) for list in train_X for word in list]
    uniquewordlist = []  # all unique words in a single list
    freqlist = []        # list of frequency of unique words
    
    for word in wordlist:
        if word not in uniquewordlist:
            uniquewordlist.append(word)
            freqlist.append(1)
        else:
            freqlist[uniquewordlist.index(word)] = freqlist[uniquewordlist.index(word)] + 1
    
    word_count["word"] = uniquewordlist
    word_count["freq"] = freqlist
    
    # TODO: Sort the words found in `data` so that sorted_words[0] is the most frequently appearing word and
    #       sorted_words[-1] is the least frequently appearing word.
    
    sorted_words = None
    sorted_freqlist, sorted_words = zip(*sorted(zip(freqlist, uniquewordlist), reverse=True))
    
    word_dict = {} # This is what we are building, a dictionary that translates words into integers
    for idx, word in enumerate(sorted_words[:vocab_size - 2]): # The -2 is so that we save room for the 'no word'
        word_dict[word] = idx + 2                              # 'infrequent' labels
        
    return word_dict

In [ ]:
# Custom - Retrieve word_dict from cache_dir if word_dict already built in previous log-in
cache_dir = '../cache/sentiment_analysis'  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

with open(os.path.join(cache_dir, 'word_dict.pkl'), "rb") as f:
    word_dict = pickle.load(f)

In [ ]:
word_dict = build_dict(train_X)

In [ ]:
# TODO: Use this space to determine the five most frequently appearing words in the training set.
freq_words = list(word_dict.keys())
print(freq_words[0:20]) 

['movi', 'film', 'one', 'like', 'time', 'good', 'make', 'charact', 'get', 'see', 'watch', 'stori', 'even', 'would', 'realli', 'well', 'scene', 'look', 'show', 'much']


### Save `word_dict`

Later on when we construct an endpoint which processes a submitted review we will need to make use of the `word_dict` which we have created. As such, we will save it to a file now for future use.

In [ ]:
# Custom - Store word_dict in cache_dir
cache_dir = '../cache/sentiment_analysis'  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

with open(os.path.join(cache_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f)         

<class 'dict'>


In [ ]:
data_dir = '../data/pytorch' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

In [ ]:
with open(os.path.join(data_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f)

### Transform the reviews

Now that we have our word dictionary which allows us to transform the words appearing in the reviews into integers, it is time to make use of it and convert our reviews to their integer sequence representation, making sure to pad or truncate to a fixed length, which in our case is `500`.

In [ ]:
def convert_and_pad(word_dict, sentence, pad=500):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1 # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad
    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:
            working_sentence[word_index] = word_dict[word]
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=500):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convert_and_pad(word_dict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)

In [ ]:
train_X, train_X_len = convert_and_pad_data(word_dict, train_X)
test_X, test_X_len = convert_and_pad_data(word_dict, test_X)

As a quick check to make sure that things are working as intended, check to see what one of the reviews in the training set looks like after having been processeed. Does this look reasonable? What is the length of a review in the training set?

In [ ]:
# Use this cell to examine one of the processed reviews to make sure everything is working as intended.
print(train_X[0])
print(train_X_len[0])
# Seems to have worked alright. The number of non-zero elements - the indices of words from the word_dict list -
# is the same as the length of the review, which is intended.

[ 304  242   98 3057   32  833  307  263  260   26   40  300    2    1
  489    2 2937 1469    6    1    1   57 3537   60    1    1  264   43
  143   53    2 1074 1142   93  169    2 2261  144  845  370    1   46
  777 1682  795   13  602    1    1   87  484 1102   55  732    1  327
   28  277 2501   10  435   18 1946 1101  697    1  602 3015    1    2
    1 2501    2  111  279 1946 1011 3206  489    1 1246    2  317 3817
    1 3817 1739    1    2   18 1114    2 4451  689  574  382  130   11
 4658    2  123    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

## Step 3: Upload the data to S3

### Save the processed training dataset locally

It is important to note the format of the data that we are saving as we will need to know it when we write the training code. In our case, each row of the dataset has the form `label`, `length`, `review[500]` where `review[500]` is a sequence of `500` integers representing the words in the review.

In [ ]:
import pandas as pd
    
pd.concat([pd.DataFrame(train_y), pd.DataFrame(train_X_len), pd.DataFrame(train_X)], axis=1) \
        .to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

### Uploading the training data


Next, we need to upload the training data to the SageMaker default S3 bucket so that we can provide access to it while training our model.

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/sentiment_rnn'

role = sagemaker.get_execution_role()

In [ ]:
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

## Step 4: Build and Train the PyTorch Model

We will start by implementing our own neural network in PyTorch along with a training script. For the purposes of this project we have provided the necessary model object in the `model.py` file, inside of the `train` folder. You can see the provided implementation by running the cell below.

In [ ]:
!pygmentize train/model.py

import torch.nn as nn

class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sig = nn.Sigmoid()
        
        self.word_dict = None

    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        x = x.t()
        lengths = x[0,:]
        reviews = x[1:,:]
        embeds = self.embedding(reviews)
        lstm_out, _ = self.lstm(embeds)
        out = self.dense(lstm_out)
        out = out[lengths - 1, range(len(lengths))]
        return self.sig

In [ ]:
import torch
import torch.utils.data

# Read in only the first 250 rows
train_sample = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None, nrows=250)

# Turn the input pandas dataframe into tensors
train_sample_y = torch.from_numpy(train_sample[[0]].values).float().squeeze()
train_sample_X = torch.from_numpy(train_sample.drop([0], axis=1).values).long()

# Build the dataset
train_sample_ds = torch.utils.data.TensorDataset(train_sample_X, train_sample_y)
# Build the dataloader
train_sample_dl = torch.utils.data.DataLoader(train_sample_ds, batch_size=50)

### Writing the training method

Next we need to write the training code itself. This should be very similar to training methods that you have written before to train PyTorch models. We will leave any difficult aspects such as model saving / loading and parameter loading until a little later.

In [ ]:
def train(model, train_loader, epochs, optimizer, loss_fn, device):
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        for batch in train_loader:         
            batch_X, batch_y = batch
            
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            # TODO: Complete this train method to train the model provided.
            output_batch = model(batch_X)  # compute model output
            loss = loss_fn(output_batch.squeeze(), batch_y)  # calculate loss
            optimizer.zero_grad()                  # clear previous gradients
            loss.backward()                        # compute gradients of all variables wrt loss
            optimizer.step()                       # perform updates using calculated gradients
            
            total_loss += loss.data.item()
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_loader)))

In [ ]:
import torch.optim as optim
from train.model import LSTMClassifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(32, 100, 5000).to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()

train(model, train_sample_dl, 5, optimizer, loss_fn, device)

Epoch: 1, BCELoss: 0.6900425314903259
Epoch: 2, BCELoss: 0.6786326766014099
Epoch: 3, BCELoss: 0.6677982568740845
Epoch: 4, BCELoss: 0.6547678947448731
Epoch: 5, BCELoss: 0.6372617363929749


### Training the model


In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="train.py",
                    source_dir="train",
                    role=role,
                    framework_version='0.4.0',
                    train_instance_count=1,
                    train_instance_type='ml.m4.xlarge', # original ml.p2.xlarge, unauthorized to request AWS
                    hyperparameters={
                        'epochs': 10,
                        'hidden_dim': 200,
                    })

In [ ]:
estimator.fit({'training': input_data})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2021-04-08 10:38:07 Starting - Starting the training job...
2021-04-08 10:38:09 Starting - Launching requested ML instances.........
2021-04-08 10:39:39 Starting - Preparing the instances for training......
2021-04-08 10:41:04 Downloading - Downloading input data...
2021-04-08 10:41:24 Training - Downloading the training image.................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-04-08 10:44:16,682 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-04-08 10:44:16,685 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2021-04-08 10:44:16,699 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-04-08 10:44:16,710 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-04-08 10:44:17,348 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
20

Epoch: 1, BCELoss: 0.6694670854782572
Epoch: 2, BCELoss: 0.6274341515132359
Epoch: 3, BCELoss: 0.5674002048920612
Epoch: 4, BCELoss: 0.49718912462798914
Epoch: 5, BCELoss: 0.43789916500753284
Epoch: 6, BCELoss: 0.39180668641109856
Epoch: 7, BCELoss: 0.3598738257982293
Epoch: 8, BCELoss: 0.32745734769470836


**Custom Note**: As seen above the training appears to have stopped at Epoch 8. Whereas, it has been completed as confirmed on Cloudwatch training logs. The notebook session had timed out which is why the display update had stopped working. It took 2 hours for the training to complete. The reason being - training instance 'ml.m4.xlarge' used instead of 'ml.p2.xlarge'. For unkown reasons I am unauthorized to create a support request on AWS support to request an instance limit increase. I had raised 2 support tickets, but the issue still hasn't been resolved. Hence, I had to improvise and move ahead with the instance available at hand.

## Step 5: Testing the model

As mentioned at the top of this notebook, we will be testing this model by first deploying it and then sending the testing data to the deployed endpoint. We will do this so that we can make sure that the deployed model is working correctly.

## Step 6: Deploy the model for testing

 Deploy the trained model.

In [ ]:
# TODO: Deploy the trained model
predictor = estimator.deploy(instance_type='ml.m4.xlarge',
                                     initial_instance_count=1)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


-----------------!

## Step 7 - Use the model for testing

Once deployed, we can read in the test data and send it off to our deployed model to get some results. Once we collect all of the results we can determine how accurate our model is.

In [ ]:
test_X = pd.concat([pd.DataFrame(test_X_len), pd.DataFrame(test_X)], axis=1)

In [ ]:
# We split the data into chunks and send each chunk seperately, accumulating the results.

def predict(data, rows=512):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = np.array([])
    for array in split_array:
        predictions = np.append(predictions, predictor.predict(array))
    
    return predictions

In [ ]:
predictions = predict(test_X.values)
predictions = [round(num) for num in predictions]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

0.84132

###  More testing

We now have a trained model which has been deployed and which we can send processed reviews to and which returns the predicted sentiment. However, ultimately we would like to be able to send our model an unprocessed review. That is, we would like to send the review itself as a string. For example, suppose we wish to send the following review to our model.

In [ ]:
test_review = 'The simplest pleasures in life are the awesome, and this film is one of them. Combining a rather basic storyline of love and adventure this movie transcends the usual weekend fair with wit and unmitigated charm.'

The question we now need to answer is, how do we send this review to our model?

Recall in the first section of this notebook we did a bunch of data processing to the IMDb dataset. In particular, we did two specific things to the provided reviews.
 - Removed any html tags and stemmed the input
 - Encoded the review as a sequence of integers using `word_dict`
 
In order process the review we will need to repeat these two steps.

Using the `review_to_words` and `convert_and_pad` methods from section one, convert `test_review` into a numpy array `test_data` suitable to send to our model. Remember that our model expects input of the form `review_length, review[500]`.

In [ ]:
# TODO: Convert test_review into a form usable by the model and save the results in test_data
test_data = None

test_words = review_to_words(test_review)
test_X_single, test_X_single_len = convert_and_pad(word_dict, test_words)
test_data = np.array([np.array([test_X_single_len] + test_X_single)]) # needs to be arrayed
#test_data = pd.concat([pd.DataFrame(test_X_single_len), pd.DataFrame(test_X_single)], axis=1) #won't work

Now that we have processed the review, we can send the resulting array to our model to predict the sentiment of the review.

In [ ]:
predictor.predict(test_data)

array(0.87232673, dtype=float32)

Since the return value of our model is close to `1`, we can be certain that the review we submitted is positive.

### Delete the endpoint

Of course, just like in the XGBoost notebook, once we've deployed an endpoint it continues to run until we tell it to shut down. Since we are done using our endpoint for now, we can delete it.

In [ ]:
estimator.delete_endpoint()

estimator.delete_endpoint() will be deprecated in SageMaker Python SDK v2. Please use the delete_endpoint() function on your predictor instead.


In [ ]:
#Store the model information so you can call 
print(estimator.model_data)
print(role)

s3://sagemaker-us-east-1-286722933665/sagemaker-pytorch-2021-04-08-10-38-07-376/output/model.tar.gz
arn:aws:iam::286722933665:role/service-role/AmazonSageMaker-ExecutionRole-20210324T230648


In [ ]:
#print(estimator.model_data)
print(role)

arn:aws:iam::286722933665:role/service-role/AmazonSageMaker-ExecutionRole-20210324T230648


## Step 6 (again) - Deploy the model for the web app

### (TODO) Writing inference code

Before writing our custom inference code, we will begin by taking a look at the code which has been provided.

In [ ]:
!pygmentize serve/predict.py

import argparse
import json
import os
import pickle
import sys
import sagemaker_containers
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data

from model import LSTMClassifier

from utils import review_to_words, convert_and_pad

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LSTMClassifier(model_info['embedding_dim'], model_info['hidden_dim'], model_info['vocab_size'])

    # Load the store model parameters.
    model_path = os.path.join(mod

### Deploying the model

Now that the custom inference code has been written, we will create and deploy our model. To begin with, we need to construct a new PyTorchModel object which points to the model artifacts created during training and also points to the inference code that we wish to use. Then we can call the deploy method to launch the deployment container.

**NOTE**: The default behaviour for a deployed PyTorch model is to assume that any input passed to the predictor is a `numpy` array. In our case we want to send a string so we need to construct a simple wrapper around the `RealTimePredictor` class to accomodate simple strings. In a more complicated situation you may want to provide a serialization object, for example if you wanted to sent image data.

In [ ]:
# Copy model data from previously created model artifact
my_model_data = 's3://sagemaker-us-east-1-286722933665/sagemaker-pytorch-2021-04-08-10-38-07-376/output/model.tar.gz'

In [ ]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.pytorch import PyTorchModel

class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

model = PyTorchModel(model_data=my_model_data,
                     role = role,
                     framework_version='0.4.0',
                     entry_point='predict.py',
                     source_dir='serve',
                     predictor_cls=StringPredictor)
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


---------------!

### Testing the model

Now that we have deployed our model with the custom inference code, we should test to see if everything is working. Here we test our model by loading the first `250` positive and negative reviews and send them to the endpoint, then collect the results. The reason for only sending some of the data is that the amount of time it takes for our model to process the input and then perform inference is quite long and so testing the entire data set would be prohibitive.

In [ ]:
print(model.model_data)
print(model.role)

s3://sagemaker-us-east-1-286722933665/sagemaker-pytorch-2021-04-08-10-38-07-376/output/model.tar.gz
arn:aws:iam::286722933665:role/service-role/AmazonSageMaker-ExecutionRole-20210324T230648


In [ ]:
import glob

def test_reviews(data_dir='../data/aclImdb', stop=250):
    
    results = []
    ground = []
    
    # We make sure to test both positive and negative reviews    
    for sentiment in ['pos', 'neg']:
        
        path = os.path.join(data_dir, 'test', sentiment, '*.txt')
        files = glob.glob(path)
        
        files_read = 0
        
        print('Starting ', sentiment, ' files')
        
        # Iterate through the files and send them to the predictor
        for f in files:
            with open(f) as review:
                # First, we store the ground truth (was the review positive or negative)
                if sentiment == 'pos':
                    ground.append(1)
                else:
                    ground.append(0)
                # Read in the review and convert to 'utf-8' for transmission via HTTP
                review_input = review.read().encode('utf-8')
                # Send the review to the predictor and store the results
                results.append(float(predictor.predict(review_input)))
                
            # Sending reviews to our endpoint one at a time takes a while so we
            # only send a small number of reviews
            files_read += 1
            if files_read == stop:
                break
            
    return ground, results

In [ ]:
ground, results = test_reviews()

Starting  pos  files
Starting  neg  files


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(ground, results)

0.83

As an additional test, we can try sending the `test_review` that we looked at earlier.

In [ ]:
predictor.predict(test_review)

b'1.0'

Now that we know our endpoint is working as expected, we can set up the web page that will interact with it. If you don't have time to finish the project now, make sure to skip down to the end of this notebook and shut down your endpoint. You can deploy it again when you come back.

## Step 7 (again): Use the model for the web app


#### Part A: Create an IAM Role for the Lambda function

#### Part B: Create a Lambda function





In [ ]:
predictor.endpoint

'sagemaker-pytorch-2021-04-09-12-10-35-384'

Once you have added the endpoint name to the Lambda function, click on **Save**. Your Lambda function is now up and running. Next we need to create a way for our web app to execute the Lambda function.

### Setting up API Gateway


## Step 4: Deploying our web app


Now that your web app is working, trying playing around with it and see how well it works.

### Delete the endpoint

Remember to always shut down your endpoint if you are no longer using it. You are charged for the length of time that the endpoint is running so if you forget and leave it on you could end up with an unexpectedly large bill.

In [ ]:
predictor.delete_endpoint()